# SSRT vegetation example


In [ ]:
import numpy as np
from mwrtms import mwRTMs
from mwrtms.scattering.volume import SSRTModel, CanopyProperties
from mwrtms.medium import VegetationMedium, SoilMedium
from mwrtms.core import ElectromagneticWave, ScatteringGeometry, PolarizationState

soil_result = mwRTMs.compute_soil_backscatter('aiem', 5.4, 40.0, 1.0, 5.0, soil_moisture=0.25)
soil_sigma0 = {pol: 10**(val/10.0) for pol, val in soil_result.items()}

wave = ElectromagneticWave(frequency_hz=5.4e9)
geometry = ScatteringGeometry(theta_i_deg=40.0)
canopy = CanopyProperties(lai=3.0, vegetation_water_content=2.5, height=1.5)
ssrt = SSRTModel(wave, geometry, canopy, soil_sigma0)
veg = VegetationMedium(gravimetric_moisture=0.6)
air = SoilMedium(0.01, 0.3, 0.5)

sigma_vv = ssrt.compute(air, veg, PolarizationState.VV)
10 * np.log10(sigma_vv + 1e-30)